In [4]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import ast
import os
from PIL import Image
import yaml

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd

# CSV 파일 로드 함수
def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

# 데이터 로드
data_bumper = load_data_with_pandas('./data/data/output/bumper/bumper_data.csv')
data_door_ed = load_data_with_pandas('./data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas('./data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas('./data/data/output/fender/fender_ed_data.csv')
data_frame_ed = load_data_with_pandas('./data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas('./data/data/output/frame/frame_hd_data.csv')
data_frame_sealf = load_data_with_pandas('./data/data/output/frame/frame_sealf_data.csv')
data_frame_seamf = load_data_with_pandas('./data/data/output/frame/frame_seamf_data.csv')

# 데이터 결합
data = pd.concat([
    data_bumper,
    data_door_ed,
    data_door_scratch,
    data_fender,
    data_frame_ed,
    data_frame_hd,
    data_frame_sealf,
    data_frame_seamf
], axis=0)

# y에는 라벨, x에는 나머지 특성 정보
y = data['quality']

# Stratified Split: 학습 데이터와 검증 데이터 비율을 80:20으로 설정, 클래스 비율을 유지
train_data, test_data = train_test_split(data, test_size=0.2, stratify=y, random_state=60)

# 학습 데이터에서 다시 80:20 비율로 나누어 검증 데이터 분리
train_data_80, valid_data = train_test_split(train_data, test_size=0.2, stratify=train_data['quality'], random_state=60)

# CSV로 저장
train_data_80.to_csv('./data/train_data.csv', index=False)
test_data.to_csv('./data/test_data.csv', index=False)
valid_data.to_csv('./data/valid_data.csv', index=False)

# 결과 출력
print(f"총 데이터 개수: {len(data)}")
print(f"학습 데이터 개수: {len(train_data_80)}")
print(f"테스트 데이터 개수: {len(test_data)}")
print(f"검증 데이터 개수: {len(valid_data)}")

총 데이터 개수: 7381
학습 데이터 개수: 4723
테스트 데이터 개수: 1477
검증 데이터 개수: 1181


In [6]:
# Step 1: 학습 데이터에서 20% 추출
train_20, _ = train_test_split(train_data_80, test_size=1, random_state=35)

# Step 2: 검증 데이터에서 20% 추출
valid_20, _ = train_test_split(valid_data, test_size=1, random_state=35)

# Step 3: 테스트 데이터에서 20% 추출
test_20, _ = train_test_split(test_data, test_size=1, random_state=35)

# Step 4: 각각의 20%에서 다시 20% 추출 (성능 검증용 데이터 생성)
train_test_20, _ = train_test_split(train_20, test_size=0.8, random_state=42)  # 학습 데이터 20%에서 20% 추출
test_test_20, _ = train_test_split(test_20, test_size=0.8, random_state=42)  # 테스트 데이터 20%에서 20% 추출
valid_test_20, _ = train_test_split(valid_20, test_size=0.8, random_state=42)  # 검증 데이터 20%에서 20% 추출

# CSV로 저장
train_test_20.to_csv('./data/train_test_20.csv', index=False)
test_test_20.to_csv('./data/test_test_20.csv', index=False)
valid_test_20.to_csv('./data/valid_test_20.csv', index=False)

In [7]:
# 결과 출력
print(f"학습 데이터 개수: {len(train_test_20)}")
print(f"테스트 데이터 개수: {len(test_test_20)}")
print(f"검증 데이터 개수: {len(valid_test_20)}")

학습 데이터 개수: 944
테스트 데이터 개수: 295
검증 데이터 개수: 236


In [8]:
# 이미지 복사
import shutil

def copy_images(data, output_dir):
    image_dir = os.path.join(output_dir, 'images')
    os.makedirs(image_dir, exist_ok=True)

    for index, row in data.iterrows():
        image_path = row['path']
        shutil.copy(image_path, image_dir)


# 학습 데이터와 검증 데이터를 각각 처리
copy_images(train_test_20, './train')
copy_images(test_test_20, './test')
copy_images(valid_test_20,'./valid')

In [9]:
import ast

def parse_bboxes(data):
    """
    문자열 형태의 bounding box 정보를 파싱하여 리스트 형태로 변환하는 함수

    Args:
        data: 파싱할 데이터셋

    Returns:
        list: 파싱된 bounding box 리스트
    """

    bboxes_list = []
    for bbox_str in data["bboxes"]:
        try:
            bboxes = ast.literal_eval(bbox_str)
            bbox_list = []
            for bbox in bboxes:
                x, y, w, h = bbox
                bbox_list.append([x, y, w, h])
            bboxes_list.append(bbox_list)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing bbox: {e}")
    return bboxes_list

# 데이터셋별 bounding box 파싱
train_bboxes = parse_bboxes(train_test_20)
valid_bboxes = parse_bboxes(valid_test_20)
test_bboxes = parse_bboxes(test_test_20)

# 결과 출력 (예시)
print(train_bboxes[:5])
print(valid_bboxes[:5])
print(test_bboxes[:5])

[[[1699.5348837209303, 454.22958280657406, 791.2611231549305, 746.4196207332488]], [], [], [], [[1966.5410206718348, 742.5374240565486, 280.94735142118867, 272.5414402967636]]]
[[], [[228.55813277612395, 709.1162790697674, 2543.441860465116, 1213.1162790697674]], [[1370.8850039968154, 934.7472938254126, 70.16865622410015, 188.74129424937706]], [[1676.0930232558135, 563.3532934131737, 990.4186046511627, 921.3173652694612]], []]
[[[1878.5195324323292, 753.7019166698774, 357.2726158219439, 261.6860578728348]], [[1693.674418604651, 41.02325581395349, 873.2093023255818, 1195.5348837209303], [2309.0232558139537, 1218.9767441860465, 416.0930232558135, 234.41860465116292], [1810.8837209302324, 1752.279069767442, 498.13953488372135, 164.09302325581393]], [], [], [[3453.4883720930234, 3505.813953488372, 6139.534883720929, 2877.906976744186]]]


In [10]:
import os
from PIL import Image


def extract_image_info(image_paths):
    """
    이미지 경로 리스트에서 이미지 크기, 디렉토리 경로, 파일 이름을 추출하는 함수

    Args:
        image_paths (list): 이미지 경로 리스트

    Returns:
        tuple: (image_sizes, image_dirs, file_names)
            image_sizes (list): 이미지 크기 리스트
            image_dirs (list): 이미지 디렉토리 경로 리스트
            file_names (list): 이미지 파일 이름 리스트
    """

    image_sizes = []
    image_dirs = []
    file_names = []

    for path in image_paths:
        try:
            image = Image.open(path)
            width, height = image.size
            image_sizes.append((width, height))
            image_dirs.append(os.path.dirname(path) + os.sep)
            file_names.append(os.path.splitext(os.path.basename(path))[0])
        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {path}")

    return image_sizes, image_dirs, file_names


# 데이터 준비 (예시)
train_image_paths = train_test_20.iloc[:, 0].tolist()
valid_image_paths = valid_test_20.iloc[:, 0].tolist()
test_image_paths = test_test_20.iloc[:, 0].tolist()

# 이미지 정보 추출
train_sizes, train_dirs, train_names = extract_image_info(train_image_paths)
valid_sizes, valid_dirs, valid_names = extract_image_info(valid_image_paths)
test_sizes, test_dirs, test_names = extract_image_info(test_image_paths)

# 결과 출력
print("Train 이미지 크기:", train_sizes[:5])
print("Train 이미지 디렉토리:", train_dirs[:5])
print("Train 파일 이름:", train_names[:5])
print()
print("Validation 이미지 크기:", valid_sizes[:5])
print("Validation 이미지 디렉토리:", valid_dirs[:5])
print("Validation 파일 이름:", valid_names[:5])
print()
print("test 이미지 크기:", test_sizes[:5])
print("test 이미지 디렉토리:", test_dirs[:5])
print("test 파일 이름:", test_names[:5])

Train 이미지 크기: [(4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000)]
Train 이미지 디렉토리: ['./data/data/source_data/frame/Hole_distortion\\', './data/data/source_data/fender/Exterior_damage\\', './data/data/source_data/bumper/scratch\\', './data/data/source_data/fender/Exterior_damage\\', './data/data/source_data/frame/Exterior_damage\\']
Train 파일 이름: ['207_213_20_82a71406-621a-4e56-aadf-3c75a739cc96', '209_102_10_d15e979f-4590-458c-b78d-24565294bf6a', '205_101_10_c7a5dec0-5164-4cb4-9c30-79fff6754977', '209_102_10_98058174-706e-484b-b37b-09fdaf7a279b', '207_102_20_d41d4a95-d332-42b5-865b-fb4ac8423b49']

Validation 이미지 크기: [(4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000), (4000, 2000)]
Validation 이미지 디렉토리: ['./data/data/source_data/bumper/scratch\\', './data/data/source_data/bumper/scratch\\', './data/data/source_data/frame/Seam_failure\\', './data/data/source_data/frame/Hole_distortion\\', './data/data/source_data/door/Exterior_damage\\']
Validation 파일 이름: ['205_101_1

In [11]:
# 부품 상태를 체크하는 함수
def get_label(part, status):
    if part == "범퍼":
        if status == 1:
            return 0
        elif status == 0:
            return 1
    elif part == "도어":
        if status == 1:
            return 0
        elif status == 0:
            return 2
    elif part == "휀더":
        if status == 1:
            return 0
        elif status == 0:
            return 3
    elif part == "프레임":
        if status == 1:
            return 0
        elif status == 0:
            return 4
    return None  # 기본적으로 반환할 값이 없으면 None


# 예시 부품과 상태
parts_and_status_train = []
parts_and_status_valid = []
parts_and_status_test = []
for quality, part in zip(train_test_20.iloc[:, 1], train_test_20.iloc[:, 2]):
    parts_and_status_train.append({"part": part, "status": quality})

for quality, part in zip(test_test_20.iloc[:, 1], test_test_20.iloc[:, 2]):
    parts_and_status_test.append({"part": part, "status": quality})

for quality, part in zip(valid_test_20.iloc[:, 1], valid_test_20.iloc[:, 2]):
    parts_and_status_valid.append({"part": part, "status": quality})

print(f"parts_and_status_train : {parts_and_status_train}")
print(f"parts_and_status_test : {parts_and_status_test}")
print(f"parts_and_status_valid : {parts_and_status_valid}")

# 라벨링 결과를 저장할 리스트
labels_train = []
labels_valid = []
label_test = []

# 각 부품 상태에 대해 라벨을 설정
for item in parts_and_status_train:
    label = get_label(item["part"], item["status"])
    labels_train.append(label)
for item in parts_and_status_valid:
    label = get_label(item["part"], item["status"])
    labels_valid.append(label)
for item in parts_and_status_test:
    label = get_label(item["part"], item["status"])
    label_test.append(label)

# # 결과 출력
print("train 라벨링 결과 :", labels_train)
print("valid 라벨링 결과 :", labels_valid)
print("test 라벨링 결과 :", label_test)

parts_and_status_train : [{'part': '프레임', 'status': 0}, {'part': '휀더', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '휀더', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '도어', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 1}, {'part': '범퍼', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '프레임', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '도어', 'status': 0}, {'part': '범퍼', 'status': 1}, {'part': '범퍼', 'status': 1}, {'part': '프레임', 'status': 0}, {'part': '프레임', 'status': 0}, {'part': '휀더', 'status': 0}, {'part': '프레임', 'status': 1}, {'part': '도어', 'status': 0}, {'part': '프레임', 'status': 0}, {'

In [12]:
def convert_to_yolo_format(bbox, label, image_width, image_height):
    yolo_labels = []  # 각 이미지에 대한 라벨을 담을 리스트

    if not bbox:
        # bbox가 비어있으면 기본값으로 라벨을 설정
        yolo_labels.append(f"{label} 0.5 0.5 0.0 0.0")
    else:
        # bbox가 있으면 여러 개의 파손 부위를 처리
        for x_min, y_min, width, height in bbox:
            # YOLO 형식에 맞게 좌표를 정규화
            x_center = (x_min + width / 2) / image_width
            y_center = (y_min + height / 2) / image_height
            norm_width = width / image_width
            norm_height = height / image_height
            # 정규화된 좌표와 함께 라벨을 저장
            yolo_labels.append(f"{label} {x_center:.6f} {y_center:.6f} {
                               norm_width:.6f} {norm_height:.6f}")

    return yolo_labels


def save_yolo_labels(output_dir, bboxes, labels, size, names):
    # 각 이미지에 대해 라벨을 텍스트 파일로 저장
    for bbox, label, (w, h), name in zip(bboxes, labels, size, names):
        yolo_labels = convert_to_yolo_format(bbox, label, w, h)

        # 라벨을 .txt 파일로 저장
        txt_file_path = os.path.join(output_dir, f"{name}.txt")
        with open(txt_file_path, 'w') as txt_file:
            for yolo_data in yolo_labels:
                txt_file.write(f"{yolo_data}\n")



# 라벨을 저장할 디렉토리
train_output_dir = './train/labels'
os.makedirs(train_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(train_output_dir, train_bboxes, labels_train, train_sizes, train_names)

# 라벨을 저장할 디렉토리
valid_output_dir = './valid/labels'
os.makedirs(valid_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(valid_output_dir, valid_bboxes, labels_valid, valid_sizes, valid_names)


# 라벨을 저장할 디렉토리
test_output_dir = './test/labels'
os.makedirs(test_output_dir, exist_ok=True)

# YOLO 라벨 저장
save_yolo_labels(test_output_dir, test_bboxes, label_test, test_sizes, test_names)

In [13]:
print(label_test)
print(len(label_test))

[4, 1, 0, 0, 1, 4, 0, 3, 0, 2, 1, 1, 2, 4, 0, 4, 0, 0, 1, 0, 0, 4, 4, 4, 0, 1, 0, 0, 2, 0, 0, 4, 0, 0, 0, 0, 4, 2, 0, 2, 0, 2, 0, 0, 4, 4, 2, 4, 4, 4, 0, 3, 2, 2, 0, 0, 1, 3, 0, 2, 0, 4, 1, 3, 4, 0, 3, 1, 0, 3, 0, 1, 0, 0, 1, 0, 1, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 0, 4, 0, 2, 0, 1, 0, 0, 4, 0, 4, 1, 0, 2, 0, 0, 0, 0, 1, 1, 0, 0, 4, 0, 0, 1, 0, 1, 0, 0, 4, 0, 4, 3, 0, 0, 0, 1, 0, 0, 1, 0, 1, 3, 0, 0, 2, 0, 4, 1, 0, 4, 1, 0, 0, 4, 1, 1, 4, 0, 0, 3, 1, 4, 2, 1, 1, 2, 0, 1, 0, 0, 0, 0, 2, 0, 4, 2, 1, 1, 1, 3, 4, 4, 4, 0, 0, 2, 2, 4, 0, 0, 0, 1, 0, 1, 4, 0, 4, 2, 2, 0, 0, 4, 4, 0, 4, 4, 2, 1, 0, 0, 0, 4, 0, 0, 4, 4, 0, 4, 4, 0, 4, 3, 4, 4, 0, 0, 3, 2, 0, 4, 4, 0, 0, 0, 4, 0, 0, 2, 3, 0, 0, 2, 0, 0, 1, 4, 1, 2, 0, 4, 1, 4, 4, 0, 1, 0, 4, 3, 2, 4, 4, 2, 1, 0, 1, 0, 4, 4, 4, 1, 2, 2, 0, 0, 2, 0, 0, 1, 0, 4, 4, 4, 1, 2, 0, 3, 0, 2, 0, 1, 1, 4, 4, 0, 4, 0, 0, 2, 1, 0, 4, 0, 0, 0, 0]
295


In [14]:
# YAML 파일에 저장
yaml_data = {
    "train": "C:/Users/Administrator/sf_project_2/train/images",
    "test": "C:/Users/Administrator/sf_project_2/test/images",
    "val": "C:/Users/Administrator/sf_project_2/valid/images",
    "nc": 5,
    "names": ["ok",
              "bumper_faulty",
              "door_faulty",
              "fender_faulty",
              "frame_faulty"]
}

with open(f"data.yaml", "w") as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False, sort_keys=False)

# print("YAML 파일이 생성되었습니다.")

In [15]:
'''
1. yolo 데이터 돌려보기
2. test데이터 만들기
3. openCV 연결 -> 실험
4. PPT 작성

'''

'\n1. yolo 데이터 돌려보기\n2. test데이터 만들기\n3. openCV 연결 -> 실험\n4. PPT 작성\n\n'

In [13]:
import os
from ultralytics import YOLO

# 데이터 설정 파일 경로
data_yaml = "data.yaml"

# YOLO 모델 정의 (랜덤 초기화로 학습 시작)

# n-s-m-l-x
model = YOLO("yolo11n.pt")  # 'yolov5n'은 작은 모델, 필요에 따라 yolov5s, yolov5m 선택 가능
# model.model = model.model.reset_parameters()  # 가중치 초기화 (랜덤 가중치)

# 학습 설정
model.train(
    data=data_yaml,           # 데이터 설정 파일 경로
    epochs=100,               # 학습 에폭 수
    batch=16,                 # 배치 크기
    imgsz=640,                # 입력 이미지 크기
    device=0,                 # GPU 사용
    workers=4,                # 데이터 로드 병렬 처리 워커 수
    project="C:/Users/Administrator/sf_project_2",     # 결과 저장 디렉토리
    name="part_damage_detection"       # 실험 이름
)

model.save('model_11_n_100_16_944.pt')

Ultralytics 8.3.64  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=C:/Users/Administrator/sf_project_2, name=part_damage_detection2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

train: Scanning C:\Users\Administrator\sf_project_2\train\labels.cache... 944 images, 0 backgrounds, 32 corrupt: 100%|██████████| 944/944 [00:00<?, ?it/s]

train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_101061f4-1ec6-4956-9e7c-5cb648fe55ce.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.104698]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_339eafbc-1b7b-4187-b6c8-42984db3a9e8.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.940407 2.934593 2.023256 2.206395]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_69849de5-024d-4509-97e7-cac881152b96.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.227767]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_6ecc616d-891b-4243-b443-10b507d0e2ca.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.768169 2.585756 2.463663]
train: WARNING  C:\Users\Administrator\sf_project_2\train\images\204_101_20_722043be-5fcb-42bc-93ac-99a0470cc111.jpg: ignoring corrupt image/la


val: Scanning C:\Users\Administrator\sf_project_2\valid\labels.cache... 236 images, 0 backgrounds, 7 corrupt: 100%|██████████| 236/236 [00:00<?, ?it/s]

val: WARNING  C:\Users\Administrator\sf_project_2\valid\images\204_101_20_b3099278-3c6b-4b51-b094-0a5638f13790.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.054884]
val: WARNING  C:\Users\Administrator\sf_project_2\valid\images\204_101_20_d0935e6c-c6f2-4f0a-ab1b-f76c23b4f0ba.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.028512]
val: WARNING  C:\Users\Administrator\sf_project_2\valid\images\205_101_20_063555ea-1697-4f3e-bcdf-d111b0f1c098.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.226834 1.931552]
val: WARNING  C:\Users\Administrator\sf_project_2\valid\images\205_101_20_0f7beccd-2c7d-405a-810b-12a168c587a4.JPG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.066605]
val: WARNING  C:\Users\Administrator\sf_project_2\valid\images\205_101_20_1b30df52-0d78-4ded-a5cb-29e597cf1bb1.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinat

...

    100/100      2.47G      1.152      1.007      1.303         16        640: 100%|██████████| 57/57 [00:17<00:00,  3.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.11it/s]

                   all        229        264      0.343      0.204        0.2     0.0835



100 epochs completed in 0.590 hours.
Optimizer stripped from C:\Users\Administrator\sf_project_2\part_damage_detection2\weights\last.pt, 5.5MB
Optimizer stripped from C:\Users\Administrator\sf_project_2\part_damage_detection2\weights\best.pt, 5.5MB

Validating C:\Users\Administrator\sf_project_2\part_damage_detection2\weights\best.pt...
Ultralytics 8.3.64  Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.04it/s]


                   all        229        264      0.381      0.208      0.214     0.0925
                    ok         95         95       0.22     0.0105     0.0143    0.00681
         bumper_faulty         31         46      0.203       0.13     0.0647     0.0213
           door_faulty         33         53      0.458      0.113      0.112     0.0504
         fender_faulty         12         12       0.31      0.167      0.191      0.056
          frame_faulty         58         58      0.715      0.621       0.69      0.328
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to C:\Users\Administrator\sf_project_2\part_damage_detection2


In [21]:

from ultralytics import YOLO

# 저장된 모델 불러오기
model = YOLO("model_8_s_50_16_944.pt")

# 추론
results = model.predict(source="C:/Users/Administrator/sf_project_2/test/images", imgsz=640)


image 1/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_0ef88ead-0d7d-4230-8bb1-7d49c063ae3f.jpg: 320x640 (no detections), 41.1ms
image 2/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_1bddf87a-2fec-45ab-9021-c9c6051939e0.jpg: 320x640 1 door_faulty, 25.4ms
image 3/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_22030eca-f5f3-4fc4-9f57-3460d45273c6.jpg: 320x640 (no detections), 16.7ms
image 4/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_63ee21f2-8e2f-436e-8065-1638d12bc38a.jpg: 320x640 (no detections), 16.6ms
image 5/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_66e438da-99ad-4ce7-bc95-96033bc6d2db.jpg: 320x640 (no detections), 18.5ms
image 6/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_74c86cc8-e672-4c38-bb9f-333ae1504582.jpg: 320x640 (no detections), 17.0ms
image 7/295 C:\Users\Administrator\sf_project_2\test\images\204_101_10_921c47c8-9786-4838-97ca-74aad33584a7.jpg: 320x640 (no detect

In [22]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

pred = []

for result in results:
    if len(result.boxes) > 0:  # Detection이 있을 경우
        # 각 box에서 confidence가 가장 높은 클래스 선택
        pred.append(int(result.boxes[0].cls))
    else:
        pred.append(0)  # No Detection은 '0'으로 처리


# y_test 생성
y_test = []

for i in range(len(test_test_20)):
    if test_test_20.iloc[i, 1] == 0:  # 열 1에 접근
        y_test.append(0)
    elif test_test_20.iloc[i, 2] == '범퍼':  # 열 2에 접근
        y_test.append(1)
    elif test_test_20.iloc[i, 2] == '도어':
        y_test.append(2)
    elif test_test_20.iloc[i, 2] == '휀더':
        y_test.append(3)
    elif test_test_20.iloc[i, 2] == '프레임':
        y_test.append(4)


## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt


# 위의 ROC 커브 코드 재사용 가능

print(y_test)
print(pred)

# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat, sep='\n')


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도:', accuracy)

# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred, average='macro')
print('정밀도: ', precision, sep='/n')


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred, average='macro')
print('재현율: ', recall)

[0, 0, 2, 1, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 4, 0, 1, 4, 0, 1, 3, 0, 0, 0, 1, 0, 2, 3, 0, 2, 4, 0, 4, 4, 1, 4, 0, 0, 4, 0, 1, 0, 4, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 4, 0, 0, 2, 0, 4, 0, 0, 0, 0, 2, 0, 0, 1, 0, 4, 0, 2, 3, 0, 1, 0, 0, 4, 1, 2, 0, 4, 4, 4, 2, 4, 0, 4, 0, 4, 0, 4, 0, 1, 1, 0, 4, 0, 0, 4, 0, 4, 4, 4, 4, 0, 0, 1, 1, 0, 2, 4, 0, 1, 0, 3, 4, 0, 1, 0, 0, 4, 2, 1, 0, 3, 4, 0, 1, 0, 0, 3, 1, 0, 1, 0, 0, 3, 0, 0, 4, 2, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 3, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 1, 4, 2, 0, 3, 0, 0, 4, 0, 0, 0, 2, 4, 0, 0, 1, 0, 0, 0, 0, 4, 2, 2, 0, 4, 4, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 4, 0, 0, 4, 0, 0, 3, 1, 4, 0, 2, 4, 0, 0, 4, 4, 0, 3, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 4, 2, 0, 3, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 4, 0, 0, 0, 0, 4, 0, 2, 4, 0, 0, 4, 0, 1, 3, 3, 2]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 2, 0, 0, 0,